In [1]:
# Setup
import sys
from pathlib import Path
import importlib

# Agregar directorio raíz al path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")

✅ Project root: D:\Academico\SENATI\octavo_ciclo\appBigData


In [2]:
# Imports y recargar módulos
from sqlalchemy import create_engine, text
from config.config import Config
from src.logger_config import get_logger
import pandas as pd

# Recargar módulos para tomar cambios
import src.validator
import src.staging

importlib.reload(src.validator)
importlib.reload(src.staging)

from src.validator import DataValidator
from src.staging import StagingProcessor

logger = get_logger('notebook_staging')
logger.info("🚀 Iniciando proceso de staging y validaciones")

✅ Configuración cargada correctamente
04:34:44 | notebook_staging | INFO | 🚀 Iniciando proceso de staging y validaciones


In [3]:
# Obtener último ETL ID
engine_staging = create_engine(Config.get_staging_connection_string())

query_etl = text("""
    SELECT etl_id, proceso, fecha_fin, estado
    FROM etl_control
    WHERE proceso LIKE 'EXTRACCION%'
    ORDER BY etl_id DESC
    LIMIT 1
""")

with engine_staging.connect() as conn:
    result = conn.execute(query_etl)
    ultimo_etl = result.fetchone()
    
    if ultimo_etl:
        etl_id = ultimo_etl[0]
        print(f"📝 Usando ETL ID: {etl_id}")
        print(f"   Proceso: {ultimo_etl[1]}")
        print(f"   Estado: {ultimo_etl[3]}")
    else:
        print("⚠️  No hay ETL previo, usando ID None")
        etl_id = None

📝 Usando ETL ID: 4
   Proceso: EXTRACCION_COMPLETA
   Estado: COMPLETADO


In [4]:
# Inicializar validador y procesador
validator = DataValidator(etl_id=etl_id)
processor = StagingProcessor(etl_id=etl_id)

logger.info("✅ Validador y procesador inicializados")

04:34:47 | validator | INFO | ✅ Validador inicializado correctamente
04:34:47 | staging | INFO | ✅ Procesador de staging inicializado
04:34:47 | notebook_staging | INFO | ✅ Validador y procesador inicializados


In [5]:
# FASE 1 - Validaciones PRE-limpieza
"""
Primero validamos los datos crudos para documentar el estado inicial
"""
print("\n" + "="*80)
print("🔍 FASE 1: VALIDACIONES PRE-LIMPIEZA")
print("="*80)

resultados_pre = validator.ejecutar_validaciones_staging()

# Mostrar resumen
print("\n📊 Resumen de validaciones PRE-limpieza:")
for validacion, resultado in resultados_pre.items():
    status = "✅" if resultado else "❌"
    print(f"{status} {validacion}")

tasa_exito_pre = sum(resultados_pre.values()) / len(resultados_pre) * 100
print(f"\n📈 Tasa de éxito: {tasa_exito_pre:.1f}%")


🔍 FASE 1: VALIDACIONES PRE-LIMPIEZA
04:34:50 | validator | INFO | ================================================================================
04:34:50 | validator | INFO | 🚀 INICIO: VALIDACIONES
04:34:50 | validator | INFO |    Validando calidad de datos en staging
04:34:50 | validator | INFO | ================================================================================
04:34:50 | validator | INFO | 🔍 Validando stg_rental...
04:34:50 | validator | INFO | ✅ Sin duplicados en stg_rental
04:34:50 | validator | INFO | ✅ Todas las columnas requeridas completas en stg_rental
04:34:50 | validator | INFO | 🔍 Validando stg_payment...
04:34:50 | validator | INFO | ✅ Sin duplicados en stg_payment
04:34:51 | validator | INFO | ✅ Todas las columnas requeridas completas en stg_payment
04:34:51 | validator | INFO | ✅ Valores en rango válido: stg_payment.amount
04:34:51 | validator | INFO | 🔍 Validando stg_film...
04:34:51 | validator | INFO | ✅ Sin duplicados en stg_film
04:34:51 | validato

In [6]:
#  Agregar columnas de validación a tablas staging
from sqlalchemy import text

print("\n" + "="*80)
print("🔧 PREPARACIÓN: Agregar columnas de validación")
print("="*80)

tablas_con_validacion = ['stg_rental', 'stg_payment', 'stg_film']

with engine_staging.connect() as conn:
    for tabla in tablas_con_validacion:
        try:
            # Agregar columna es_valido
            query1 = text(f"""
                ALTER TABLE {tabla}
                ADD COLUMN es_valido BOOLEAN DEFAULT TRUE
            """)
            conn.execute(query1)
            conn.commit()
            print(f"✅ Agregada columna es_valido a {tabla}")
        except Exception as e:
            if '1060' in str(e):  # Columna ya existe
                print(f"⚠️  {tabla}.es_valido ya existe")
            else:
                print(f"❌ Error en {tabla}.es_valido: {e}")
        
        try:
            # Agregar columna mensaje_validacion
            query2 = text(f"""
                ALTER TABLE {tabla}
                ADD COLUMN mensaje_validacion VARCHAR(255) NULL
            """)
            conn.execute(query2)
            conn.commit()
            print(f"✅ Agregada columna mensaje_validacion a {tabla}")
        except Exception as e:
            if '1060' in str(e):  # Columna ya existe
                print(f"⚠️  {tabla}.mensaje_validacion ya existe")
            else:
                print(f"❌ Error en {tabla}.mensaje_validacion: {e}")

print("\n✅ Columnas de validación agregadas")


🔧 PREPARACIÓN: Agregar columnas de validación
✅ Agregada columna es_valido a stg_rental
✅ Agregada columna mensaje_validacion a stg_rental
✅ Agregada columna es_valido a stg_payment
✅ Agregada columna mensaje_validacion a stg_payment
✅ Agregada columna es_valido a stg_film
✅ Agregada columna mensaje_validacion a stg_film

✅ Columnas de validación agregadas


In [7]:
# FASE 2 - Limpieza y Transformaciones
"""
Procesamos las tablas para limpiar y transformar datos
"""
print("\n" + "="*80)
print("🔧 FASE 2: LIMPIEZA Y TRANSFORMACIONES")
print("="*80)

stats_procesamiento = processor.procesar_todas_las_tablas()

# Mostrar estadísticas
print("\n📊 Estadísticas de procesamiento:")
for tabla, stats in stats_procesamiento.items():
    print(f"\n{tabla}:")
    for metric, valor in stats.items():
        if valor > 0:
            print(f"   {metric}: {valor}")


🔧 FASE 2: LIMPIEZA Y TRANSFORMACIONES
04:35:03 | staging | INFO | ================================================================================
04:35:03 | staging | INFO | 🚀 INICIO: PROCESAMIENTO_STAGING
04:35:03 | staging | INFO |    Limpieza y transformaciones en staging
04:35:03 | staging | INFO | ================================================================================
04:35:03 | staging | INFO | 🔧 Procesando stg_rental...
04:38:21 | staging | INFO | ✅ Sin duplicados en stg_rental
04:38:21 | staging | INFO | ✅ stg_rental procesado: {'duplicados': 0, 'fecha_invalida': 0, 'return_invalido': 0}
04:38:21 | staging | INFO | 🔧 Procesando stg_payment...
04:41:37 | staging | INFO | ✅ Sin duplicados en stg_payment
04:41:37 | staging | INFO | ✅ stg_payment procesado: {'duplicados': 0, 'monto_negativo': 0, 'monto_excesivo': 0, 'fecha_invalida': 0}
04:41:37 | staging | INFO | 🔧 Procesando stg_film...
04:41:38 | staging | INFO | ✅ Sin duplicados en stg_film
04:41:38 | staging | INFO 

In [8]:
# FASE 3 - Validaciones POST-limpieza
"""
Validamos nuevamente para verificar mejoras
"""
print("\n" + "="*80)
print("🔍 FASE 3: VALIDACIONES POST-LIMPIEZA")
print("="*80)

resultados_post = validator.ejecutar_validaciones_staging()

# Mostrar resumen
print("\n📊 Resumen de validaciones POST-limpieza:")
for validacion, resultado in resultados_post.items():
    status = "✅" if resultado else "❌"
    cambio = ""
    if validacion in resultados_pre:
        if resultado and not resultados_pre[validacion]:
            cambio = " (MEJORÓ ↑)"
        elif not resultado and resultados_pre[validacion]:
            cambio = " (EMPEORÓ ↓)"
    print(f"{status} {validacion}{cambio}")

tasa_exito_post = sum(resultados_post.values()) / len(resultados_post) * 100
print(f"\n📈 Tasa de éxito POST: {tasa_exito_post:.1f}%")
print(f"📈 Mejora: {tasa_exito_post - tasa_exito_pre:+.1f}%")


🔍 FASE 3: VALIDACIONES POST-LIMPIEZA
04:42:48 | validator | INFO | ================================================================================
04:42:48 | validator | INFO | 🚀 INICIO: VALIDACIONES
04:42:48 | validator | INFO |    Validando calidad de datos en staging
04:42:48 | validator | INFO | ================================================================================
04:42:48 | validator | INFO | 🔍 Validando stg_rental...
04:42:48 | validator | INFO | ✅ Sin duplicados en stg_rental
04:42:49 | validator | INFO | ✅ Todas las columnas requeridas completas en stg_rental
04:42:49 | validator | INFO | 🔍 Validando stg_payment...
04:42:49 | validator | INFO | ✅ Sin duplicados en stg_payment
04:42:49 | validator | INFO | ✅ Todas las columnas requeridas completas en stg_payment
04:42:49 | validator | INFO | ✅ Valores en rango válido: stg_payment.amount
04:42:49 | validator | INFO | 🔍 Validando stg_film...
04:42:49 | validator | INFO | ✅ Sin duplicados en stg_film
04:42:49 | validat

In [9]:
# Ver registros inválidos en detalle
"""
Examinar qué registros fueron marcados como inválidos
"""
print("\n" + "="*80)
print("📋 REGISTROS INVÁLIDOS DETECTADOS")
print("="*80)

# Rental inválidos
query_rental_inv = """
    SELECT rental_id, rental_date, return_date, mensaje_validacion
    FROM stg_rental
    WHERE es_valido = FALSE
    LIMIT 10
"""

df_rental_inv = pd.read_sql(query_rental_inv, engine_staging)
if len(df_rental_inv) > 0:
    print(f"\n⚠️  stg_rental - {len(df_rental_inv)} registros inválidos (mostrando 10):")
    display(df_rental_inv)
else:
    print("\n✅ stg_rental - Todos los registros son válidos")

# Payment inválidos
query_payment_inv = """
    SELECT payment_id, amount, payment_date, mensaje_validacion
    FROM stg_payment
    WHERE es_valido = FALSE
    LIMIT 10
"""

df_payment_inv = pd.read_sql(query_payment_inv, engine_staging)
if len(df_payment_inv) > 0:
    print(f"\n⚠️  stg_payment - {len(df_payment_inv)} registros inválidos (mostrando 10):")
    display(df_payment_inv)
else:
    print("\n✅ stg_payment - Todos los registros son válidos")


📋 REGISTROS INVÁLIDOS DETECTADOS

✅ stg_rental - Todos los registros son válidos

✅ stg_payment - Todos los registros son válidos


In [10]:
# Ver auditoría de calidad
"""
Revisar todas las validaciones registradas en audit_calidad
"""
query_audit = text("""
    SELECT 
        validacion,
        resultado,
        tabla_origen,
        COUNT(*) as veces
    FROM audit_calidad
    WHERE etl_id = :etl_id
    GROUP BY validacion, resultado, tabla_origen
    ORDER BY resultado DESC, validacion
""")

with engine_staging.connect() as conn:
    df_audit = pd.read_sql(query_audit, conn, params={"etl_id": etl_id})

print("\n📋 Auditoría de Calidad:")
display(df_audit)


📋 Auditoría de Calidad:


,validacion,resultado,tabla_origen,veces
0,Duplicados PK,PASS,stg_rental,3
1,Duplicados PK,PASS,stg_payment,3
2,Duplicados PK,PASS,stg_film,3
3,Integridad Referencial,PASS,stg_rental,3
4,Integridad Referencial,PASS,stg_inventory,3
5,Integridad Referencial,PASS,stg_store,3
6,Nulos en amount,PASS,stg_payment,3
7,Nulos en customer_id,PASS,stg_rental,3
8,Nulos en customer_id,PASS,stg_payment,3
9,Nulos en inventory_id,PASS,stg_rental,3


In [11]:
# Obtener datos limpios para siguiente fase
"""
Extraer solo registros válidos para la fase de transformación
"""
print("\n" + "="*80)
print("📦 DATOS LIMPIOS LISTOS PARA TRANSFORMACIÓN")
print("="*80)

# Contar registros válidos por tabla
tablas_staging = ['stg_rental', 'stg_payment', 'stg_film', 'stg_inventory',
                 'stg_category', 'stg_store', 'stg_city', 'stg_country']

print("\n📊 Registros válidos por tabla:")
for tabla in tablas_staging:
    df_validos = processor.obtener_registros_validos(tabla)
    print(f"   {tabla}: {len(df_validos):,} registros")


📦 DATOS LIMPIOS LISTOS PARA TRANSFORMACIÓN

📊 Registros válidos por tabla:
04:43:18 | staging | INFO | 📊 16,044 registros válidos en stg_rental
   stg_rental: 16,044 registros
04:43:18 | staging | INFO | 📊 16,044 registros válidos en stg_payment
   stg_payment: 16,044 registros
04:43:18 | staging | INFO | 📊 1,000 registros válidos en stg_film
   stg_film: 1,000 registros
04:43:18 | staging | INFO | 📊 4,581 registros válidos en stg_inventory
   stg_inventory: 4,581 registros
04:43:18 | staging | INFO | 📊 16 registros válidos en stg_category
   stg_category: 16 registros
04:43:18 | staging | INFO | 📊 2 registros válidos en stg_store
   stg_store: 2 registros
04:43:18 | staging | INFO | 📊 600 registros válidos en stg_city
   stg_city: 600 registros
04:43:18 | staging | INFO | 📊 109 registros válidos en stg_country
   stg_country: 109 registros


In [12]:
# Cerrar conexiones
validator.cerrar_conexion()
processor.cerrar_conexion()
logger.info("✅ Proceso de staging completado")

print("\n" + "="*80)
print("🎉 ¡STAGING Y VALIDACIONES COMPLETADAS!")
print("="*80)
print("\n📌 Siguiente paso: 03_transformacion.ipynb (Crear modelo estrella)")

04:43:29 | validator | INFO | 🔌 Conexión cerrada
04:43:29 | staging | INFO | 🔌 Conexión cerrada
04:43:29 | notebook_staging | INFO | ✅ Proceso de staging completado

🎉 ¡STAGING Y VALIDACIONES COMPLETADAS!

📌 Siguiente paso: 03_transformacion.ipynb (Crear modelo estrella)
